In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))


%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import pprint
import keras.backend as KB
sys.path.append('../')

import mrcnn.model        as modellib
import mrcnn.visualize    as visualize
# import mrcnn.new_shapes   as shapes
import mrcnn.new_shapes  as new_shapes
# from mrcnn.new_shapes  import NewShapesDataset, NewShapesConfig
from mrcnn.config      import Config
from mrcnn.model       import log
from mrcnn.dataset     import Dataset 
# from mrcnn.pc_prototype import PCTensor
# from mrcnn.pcn_layer    import PCNLayer, PCILayer

# Root directory of the project
ROOT_DIR = os.getcwd()
MODEL_PATH = 'E:\Models'
# Directory to save logs and trained model
MODEL_DIR = os.path.join(MODEL_PATH, "mrcnn_logs")
# Path to COCO trained weights
COCO_MODEL_PATH   = os.path.join(MODEL_PATH, "mask_rcnn_coco.h5")
RESNET_MODEL_PATH = os.path.join(MODEL_PATH, "resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5")

print("Tensorflow Version: {}   Keras Version : {} ".format(tf.__version__,keras.__version__))
pp = pprint.PrettyPrinter(indent=2, width=100)
np.set_printoptions(linewidth=100)

# Build configuration object -----------------------------------------------
config = new_shapes.NewShapesConfig()
config.BATCH_SIZE      = 2                    #Batch size is 2 (# GPUs * images/GPU).
config.IMAGES_PER_GPU  = 2
config.STEPS_PER_EPOCH = 7
config.IMAGES_PER_GPU  = 1
config.display() 

# Build shape dataset        -----------------------------------------------

# from mrcnn.datagen import data_generator, load_image_gt

# Training dataset generate 500 shapes 
dataset_test = new_shapes.NewShapesDataset()
dataset_test.load_shapes(500, config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1])
dataset_test.prepare()

import pprint
pp = pprint.PrettyPrinter(indent=4)

class InferenceConfig(new_shapes.NewShapesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()
inference_config.display() 

# Configurations

# import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
# tf_config = tf.ConfigProto()
# tf_config.gpu_options.per_process_gpu_memory_fraction = 0.55
# set_session(tf.Session(config=tf_config))

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.55)
set_session(tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)))

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)
print(' Build complete')

In [ ]:
# for layer in model.keras_model.layers
#     print layer.name 
mdl = model.keras_model
layers    = [layer for layer in mdl.layers]          # all layer outputs
len(layers)

In [ ]:
idx = 1
print(' layer {} :  {}'.format(idx,layers[idx]))
pp.pprint(len(layers[idx]._inbound_nodes))
pp.pprint(layers[idx].__dict__)
print('-------------------------------------------')

In [ ]:
for idx,layer in enumerate(layers):
    print('>layer {} : name : {:40s}  type: {}'.format(idx,layer.name,layer))
#     pp.pprint(layer._inbound_nodes[0].__dict__)

In [ ]:

for idx,layer in enumerate(layers):
    print('>layer {} : name : {}  type: {}'.format(idx,layer.name,layer))
#     pp.pprint(layer._inbound_nodes[0].__dict__)
    print(">> input_shape {} inbound node {} ".format(layer._inbound_nodes[0].input_shapes,layer._inbound_nodes[0]))
#     print('>> inbound node output_shape {}'.format(layer._inbound_nodes[0].output_shapes))
#     if isinstance("layer", mrcnn.model.BatchNorm):
#         print('Batch Nomraliation layer')
    if 'padding' in layer.__dict__:
        print('\t padding : ',layer.padding)
    if 'filters' in layer.__dict__:
        print('\t Filters : ',layer.filters, ' Kernel Size :', layer.kernel_size, 'Stride:', layer.strides, ' Shape',layer.kernel.shape)
    
    if 'activation' in layer.__dict__:
        print('\t ',layer.activation)
    if 'pool_size' in layer.__dict__:
        print('\t poolsize : ',layer.pool_size,'Stride:', layer.strides)

#     pp.pprint(layer.__dict__)
    print("<< outbound shape{} input_shape {}".format(layer._outbound_nodes[0].input_shapes,layer._outbound_nodes[0]))
#     print('<< outbound node output_shape {}'.format(layer._outbound_nodes[0].output_shapes))
    print('-------------------------------------------')
    #layer.name, layer.input._keras_shape, layer.output._keras_shap

In [ ]:
import pprint as pp
import keras

# for layer in model.keras_model.layers
#     print layer.name 
mdl = model.keras_model
print(' Total number of layers ', len(model.keras_model.layers), type(mdl))

print('\n Inputs')    
for i, inp in enumerate(mdl.input):
    print(' {}   {:35s}  {}  '.format(i, inp.name, inp.shape))
    
print('\n Outputs')    
for i, outp in enumerate(mdl.output):
    print(' {}   {:35s}  {}  '.format(i, outp.name, outp.shape))


# pp = pprint.PrettyPrinter()
for idx,layer in enumerate(mdl.layers): 
    if idx > 25  : 
        break
    print()    
    print('#'*100)
    print('#  Layer {} :  Name : {}  Type: {}'.format(idx,layer.name,layer))
    print('#'*100)
    if isinstance(layer, keras.engine.training.Model):
        print('#  MODEL LAYER ')    
#         print('\n Inputs')    
#         for i, inp in enumerate(layer.input):
#             print(' {}   {:35s}  {}  '.format(i, inp.name, inp.shape))

#         print('\n Outputs')    
#         for i, outp in enumerate(layer.output):
#             print(' {}   {:35s}  {}  '.format(i, outp.name, outp.shape))
            
    pp.pprint(layer.__dict__)
    num_inbound  = len(layer._inbound_nodes)
    print('  >>>','-'*85)   
    print('  >>>  Number of inbound nodes ', num_inbound)    
    print('  >>>','-'*85)   
    if num_inbound > 0:

        for i, ib_node in enumerate(layer._inbound_nodes):
            print('\t','-'*70)
            print("\t >> {}  {}  ".format(i, ib_node))            
            for j in ib_node.input_tensors:
                print('\t    incoming tensor(s): {:25s}  shape {}'.format(j.name, j.shape))
            print('\t','-'*70)
                
            pp.pprint(ib_node.__dict__, indent=10, width=140)
    #       print('<< outbound node output_shape {}'.format(layer._outbound_nodes[0].output_shapes))
    
    

#     if isinstance("layer", model.Keras_BatchNorm):
#         print('Batch Nomraliation layer')
    if 'padding' in layer.__dict__:
        print('\t padding : ',layer.padding)
    if 'filters' in layer.__dict__:
        print('\t Filters : ',layer.filters, ' Kernel Size :', layer.kernel_size, 'Stride:', layer.strides, ' Shape',layer.kernel.shape)
    
    if 'activation' in layer.__dict__:
        print('\t ',layer.activation)
    if 'pool_size' in layer.__dict__:
        print('\t poolsize : ',layer.pool_size,'Stride:', layer.strides)
        
    num_outbound = len(layer._outbound_nodes)
    print('  <<<','-'*85)       
    print('  <<<   Number of outbound nodes ', num_outbound)
    print('  <<<','-'*85)       
#     pp.pprint(layer.__dict__)
    if num_outbound > 0:

        for i, ob_node in enumerate(layer._outbound_nodes):
            print("<< outbound {} input_shape {}".format(i,ob_node.input_shapes))
            for j in ob_node.input_tensors:
                print(' from layer: ', j.name, j.shape)
    #         for j in ob_node.outbound_layer:
    #             print(' to layer: ',j.name)
            pp.pprint(ob_node.__dict__, indent=10, width=140)
    #       print('<< outbound node output_shape {}'.format(layer._outbound_nodes[0].output_shapes))
        
    #layer.name, layer.input._keras_shape, layer.output._keras_shape